In [1]:
import pandas as pd
import numpy as np
import os
import yfinance as yf
from datetime import timedelta

In [59]:
SPX_Prices = yf.download('SPY', start='2020-6-30', end='2023-11-01', interval = "1d")
SPX_Prices

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-06-30,303.989990,310.200012,303.820007,308.359985,292.539642,113394800
2020-07-01,309.570007,311.890015,309.070007,310.519989,294.588806,72396500
2020-07-02,314.239990,315.700012,311.510010,312.230011,296.211121,69344200
2020-07-06,316.369995,317.679993,315.559998,317.049988,300.783752,61713800
2020-07-07,315.380005,317.519989,313.369995,313.779999,297.681610,82910000
...,...,...,...,...,...,...
2023-10-25,421.890015,421.920013,417.019989,417.549988,415.863892,94223200
2023-10-26,416.450012,417.329987,411.600006,412.549988,410.884094,115156800
2023-10-27,414.190002,414.600006,409.209991,410.679993,409.021637,107367700


In [61]:
def garman_klass_volatility(data):
    """
    Calculate Garman-Klass volatility for given price data.
    """
    log_hl = np.log(data['High'] / data['Low'])
    log_co = np.log(data['Close'] / data['Open'])
    ans = 0.5 * log_hl**2 - (2 * np.log(2) - 1) * log_co**2
    return np.sqrt(252 * (np.sum(ans) * (1 / (len(data) - 1))))

def calculate_periodic_volatility(df, freq):


    period_ranges = df.resample(freq).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'}).index
    

    volatilities = []


    for start_date in period_ranges:
  
        if freq == 'W':
            end_date = start_date + pd.DateOffset(weeks=1) - pd.DateOffset(days=1)
        elif freq == 'M':
            end_date = start_date + pd.DateOffset(months=1) - pd.DateOffset(days=1)
        elif freq == 'Q':
            end_date = start_date + pd.DateOffset(months=3) - pd.DateOffset(days=1)
        

        period_data = df[start_date:end_date]
        

        if len(period_data) > 1:
            vol = garman_klass_volatility(period_data)
            volatilities.append((start_date, vol))
    

    volatility_df = pd.DataFrame(volatilities, columns=['Period Start', 'Volatility'])
    
    return volatility_df

weekly_vol = calculate_periodic_volatility(SPX_Prices, 'W')
monthly_vol = calculate_periodic_volatility(SPX_Prices, 'M')
quarterly_vol = calculate_periodic_volatility(SPX_Prices, 'Q')


In [62]:
weekly_vol

,Period Start,Volatility
0,2020-07-05,0.164406
1,2020-07-12,0.188096
2,2020-07-19,0.129412
3,2020-07-26,0.140304
4,2020-08-02,0.067319
...,...,...
169,2023-10-01,0.168128
170,2023-10-08,0.135861
171,2023-10-15,0.133170
172,2023-10-22,0.140544


In [63]:
monthly_vol

,Period Start,Volatility
0,2020-06-30,0.141574
1,2020-07-31,0.090693
2,2020-08-31,0.208867
3,2020-09-30,0.166277
4,2020-10-31,0.143877
5,2020-11-30,0.137718
6,2020-12-31,0.140435
7,2021-01-31,0.138819
8,2021-02-28,0.172370
9,2021-03-31,0.074742


In [54]:
quarterly_vol

,Period Start,Volatility
0,2020-09-30,0.009430
1,2020-12-31,0.009280
2,2021-03-31,0.005113
3,2021-06-30,0.005577
4,2021-09-30,0.007211
5,2021-12-31,0.012059
6,2022-03-31,0.013111
7,2022-06-30,0.010156
8,2022-09-30,0.012245
9,2022-12-31,0.009401
